In [1]:
from gurobipy import GRB
import gurobipy as gb


# QUESTION 1

## Extracting the necessary data

In [70]:
import pandas as pd

# Load data files
file_paths = {
    "direct_capacity": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Capacity_for_Direct_Production_Facilities.csv",
    "transship_capacity_centers": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Capacity_for_Transship_Distribution_Centers.csv",
    "transship_capacity_facilities": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Capacity_for_Transship_Production_Facilities.csv",
    "cost_production_refinement": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Cost_Production_to_Refinement.csv",
    "cost_production_transshipment": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Cost_Production_to_Transshipment.csv",
    "cost_transshipment_refinement": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Cost_Transshipment_to_Refinement.csv",
    "refinement_demand": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Refinement_Demand.csv",
}

# Reading the data
data = {key: pd.read_csv(file_paths[key]) for key in file_paths}

# Displaying the keys to understand the structure of the data
data.keys()  


dict_keys(['direct_capacity', 'transship_capacity_centers', 'transship_capacity_facilities', 'cost_production_refinement', 'cost_production_transshipment', 'cost_transshipment_refinement', 'refinement_demand'])

In [64]:
# Displaying the first few rows of each dataset to understand their structure
data_overview = {key: data[key].head() for key in data}

data_overview


{'direct_capacity':    ProductionFacility  Capacity
 0                   1       462
 1                   2       103
 2                   3       460
 3                   4       325
 4                   5       227,
 'transship_capacity_centers':    TransshipmentHub  Capacity
 0                 1      1317
 1                 2      1453,
 'transship_capacity_facilities':    ProductionFacility  Capacity
 0                   1       374
 1                   2       444
 2                   3       395
 3                   4       245
 4                   5       378,
 'cost_production_refinement':    ProductionFacility  RefinementCenter      Cost
 0                   1                 1  4.252733
 1                   1                 2  4.567726
 2                   1                 3  4.696484
 3                   1                 4  2.678741
 4                   1                 5  4.272451,
 'cost_production_transshipment':    ProductionFacility  TransshipmentHub      Cost
 0   

In [78]:
data['refinement_demand']

,RefinementCenter,Demand
0,1,1537
1,2,1748
2,3,1940
3,4,1838
4,5,1665


In [76]:
# Extracting the necessary data

# Capacities array
cap_direct = data['direct_capacity']['Capacity'].values  
cap_transship_facilities = data['transship_capacity_facilities']['Capacity'].values
cap_transship_centers = data['transship_capacity_centers']['Capacity'].values

# Costs dataframe
cost_prod_ref = data['cost_production_refinement']
cost_prod_trans = data['cost_production_transshipment']
cost_trans_ref = data['cost_transshipment_refinement']



# Number of facilities and centers
num_direct_facilities = len(cap_direct)
num_transship_facilities = len(cap_transship_facilities)
num_refinement_centers = 5  # Given in the problem statement
num_transship_centers = len(cap_transship_centers)

# demand 
demand_data = data["refinement_demand"]

print('Capacity of direct facilities:', cap_direct)
print('Capacity of transship facilities:', cap_transship_facilities)
print('Capacity of distribution center:',cap_transship_centers)


print('\n')


print('number of direct facility:', num_direct_facilities)
print('number of transship facility:', num_transship_facilities)
print('number of distirbution center', num_transship_centers)
print('number of refinement center:',num_refinement_centers)

print('\n')

print('Demand:', demand_data)



Capacity of direct facilities: [462 103 460 325 227 217 205 521 548 191 361 411 104 155 285 109 422 438
 501 139 462 504 106 132 298]
Capacity of transship facilities: [374 444 395 245 378 408 435 175 415 503 184 297 450 169 365]
Capacity of distribution center: [1317 1453]


number of direct facility: 25
number of transship facility: 15
number of distirbution center 2
number of refinement center: 5


Demand:    RefinementCenter  Demand
0                 1    1537
1                 2    1748
2                 3    1940
3                 4    1838
4                 5    1665


## Create the optimization model

In [53]:
from gurobipy import GRB, Model

# Create the optimization model
model = Model("Transshipment Problem")


## Define decision variables

In [55]:
# Define the indices for the facilities and centers
direct_facilities = range(num_direct_facilities)  
transship_facilities = range(num_transship_facilities)  
refinement_centers = range(num_refinement_centers)  
transship_centers = range(num_transship_centers)  

# Create decision variables
# Xij - Quantity from direct production facility i to refinement center j
X = model.addVars(direct_facilities, refinement_centers,lb =0, vtype=GRB.CONTINUOUS, name="X")

# Yij - Quantity from transshipment production facility i to transshipment center j
Y = model.addVars(transship_facilities, transship_centers,lb=0, vtype=GRB.CONTINUOUS, name="Y")

# Zjk - Quantity from transshipment center j to refinement center k
Z = model.addVars(transship_centers, refinement_centers,lb=0, vtype=GRB.CONTINUOUS, name="Z")

## Define objective function

In [60]:
#使得我们能够准确地从一个结构化的数据表中提取出对应每条运输路线的成本数据。
# Objective function: Minimize total cost
model.setObjective(gb.quicksum(cost_prod_ref.iloc[i * num_refinement_centers + j]['Cost'] * X[i, j] 
                               for i in range(num_direct_facilities) 
                               for j in range(num_refinement_centers)) +
                   gb.quicksum(cost_prod_trans.iloc[i * num_transship_centers + j]['Cost'] * Y[i, j] 
                               for i in range(num_transship_facilities) 
                               for j in range(num_transship_centers)) +
                   gb.quicksum(cost_trans_ref.iloc[j * num_refinement_centers + k]['Cost'] * Z[j, k] 
                               for j in range(num_transship_centers) 
                               for k in range(num_refinement_centers)),
                   GRB.MINIMIZE)




## Define constraint

In [77]:
# Constraints
# Capacity constraints for direct production facilities
for i in range(num_direct_facilities):
    model.addConstr(gb.quicksum(X[i, j] for j in range(num_refinement_centers)) <= cap_direct[i])

# Capacity constraints for transshipment production facilities
for i in range(num_transship_facilities):
    model.addConstr(gb.quicksum(Y[i, j] for j in range(num_transship_centers)) <= cap_transship_facilities[i])

# Demand constraints at each refinement center
for j in range(num_refinement_centers):
    demand_j = demand_data.loc[j, 'Demand']  
    model.addConstr(gb.quicksum(X[i, j] for i in range(num_direct_facilities)) +
                    gb.quicksum(Z[k, j] for k in range(num_transship_centers)) == demand_j)


## Optimize model

In [79]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 85 rows, 330 columns and 445 nonzeros
Model fingerprint: 0x626a2680
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 85 rows and 330 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7267260e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.726726025e+04


## Question(a): After solving the linear program, what is the optimal transportation cost?

In [80]:
# Check if the solution is found
if model.status == GRB.Status.OPTIMAL:
    optimal_cost = model.objVal
    print(f"Optimal Transportation Cost: {optimal_cost}")
else:
    print("No optimal solution found")

Optimal Transportation Cost: 17267.26024720385


Question(b): In the optimal solution, what proportion of canola oil is transshipped?

In [82]:
# 计算总的转运量
total_transshipped = sum(Y[i, j].X for i in transship_facilities for j in transship_centers) + \
                     sum(Z[j, k].X for j in transship_centers for k in refinement_centers)

# 计算总的运输量
total_transport = total_transshipped + \
                  sum(X[i, j].X for i in direct_facilities for j in refinement_centers)

# 计算比例
proportion_transshipped = total_transshipped / total_transport if total_transport > 0 else 0

print('% of transshipped:',proportion_transshipped)


% of transshipped: 0.9470669110907425



 Question(c): The model does not currently limit that amount of canola oil that is transshipped. How would you modify the objective function to account for this? Formulate and solve this model.

In [97]:
# 计算所有转运中心的总容量
cap_transship_centers = data['transship_capacity_centers']['Capacity'].values
total_transship_center_capacity = sum(cap_transship_centers)

# 将转运量的上限设定为转运中心能力的 50%
#max_transship_amount = 0.5 * total_transship_center_capacity
max_transship_amount = 0.2 * total_demand


# Constraints
# 添加转运量的约束
model.addConstr(sum(Y[i, j] for i in transship_facilities for j in transship_centers) +
                sum(Z[j, k] for j in transship_centers for k in refinement_centers) <= max_transship_amount)

# Capacity constraints for direct production facilities
for i in range(num_direct_facilities):
    model.addConstr(gb.quicksum(X[i, j] for j in range(num_refinement_centers)) <= cap_direct[i])

# Capacity constraints for transshipment production facilities
for i in range(num_transship_facilities):
    model.addConstr(gb.quicksum(Y[i, j] for j in range(num_transship_centers)) <= cap_transship_facilities[i])

# Demand constraints at each refinement center
for j in range(num_refinement_centers):
    demand_j = demand_data.loc[j, 'Demand']  
    model.addConstr(gb.quicksum(X[i, j] for i in range(num_direct_facilities)) +
                    gb.quicksum(Z[k, j] for k in range(num_transship_centers)) == demand_j)
    
model.optimize()

print('\n')
# Check if the solution is found
if model.status == GRB.Status.OPTIMAL:
    optimal_cost = model.objVal
    print(f"Optimal Transportation Cost: {optimal_cost}")
else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 639 rows, 330 columns and 4485 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 3e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8501078e+04   1.121200e+04   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible model


No optimal solution found


In [98]:
model.computeIIS()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads


IIS computed: 31 constraints and 30 bounds
IIS runtime: 0.00 seconds (0.00 work units)
